<a href="https://colab.research.google.com/github/tiagosardi/optimizationMethod/blob/main/algoritmosGulosos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/tiagosardi/optimizationMethod.git
#import sys
#sys.path.append('/content/optimizationMethod/')

Cloning into 'optimizationMethod'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 43 (delta 18), reused 14 (delta 4), pack-reused 0
Unpacking objects: 100% (43/43), done.


Na busca exaustiva ele gera todas as buscas possíveis e fica com a melhor. A vantagem da busca exautiva a gente garante que estamos na melhor solução. A desvantagem está no custo. Outra vantagem é a simplicidade (resolvemos o caixeiro viajante com busca exaustiva pois resolvemos com permutação, sem ciclos e sem restrição), então para problemas pequenos pode ser mais útil usar busca exaustiva. O problema é quando a instância vai crescendo e a busca exaustiva vai se distanciando do modelo mais adequado.

Vamos usar métodos heurísticos desta vez. Heurística é um procedimento aproximado. Não tem garantia de encontrar o ótimo, mas encontraremos uma solução boa com mais rapidez. Existem dois tipos básicos de heurísticas: As construtivas e a heurísticas de refinamento.

As construtivas servem pra gente construir uma solução boa. A rota inicial tem que ser melhor que a aleatória.

Na heuristica de refinamento, nós refinamos uma solução que já temos. 
ENtão combinaremos os dois tipos. Primeiro vamos usar a construtiva e depois refinamento.

A heuristica construtiva mais comum é o algoritmo guloso.

Precisamos primeiro definir a gula desse algoritmo usando uma função h:Z -> R (a funçao mapeia numeros inteiros num numero real)

Cada iteração o algoritmo escolhe um elemento pra adicionar a solução que tem o maior valor dessa heuristica possível.
ENtão temos o método heuristico e a função heurística.
A funçao vai definir qual a fome desse algoritmo. 

O PROBLEMA DA MOCHILA:
(Knapsack problem)

Nós temos um conjunto de itens I e para cada item nós temos um lucro P (profit) e temos um peso W (weight). 
Cada item que colocamos na mochila, traz um lucro ao ser vendido. Os itens tem pesos e a mochila tem um limite de peso, onde C é a capacidade da mochila.
Precisamos maximizar o lucro e manter a capacidade.
Vamos modelar esse problema como um problema de programação linear:

Temos um vetor de pesos $\bar{P}$ = [40,35,18,4,10,2] e um vetor de pesos $\bar{W}$ = [100,50,45,20,10,5].
Agora, para formular como problema de programação linear, vamos olhar para as variáveis que existem no problema. Não podemos mudar o peso dos itens, mas variamos em quais itens poderemos pegar:

$ Xi \in{\{0,1\}}$ -> indica se o item i está na mochila.

$\bar{x} = [0,0,1,0,1,0]$ Temos 6 itens, então um vetor com 6 posições, onde colocamos os itens 2 e 4 na mochila e os demais não foram adicionados a mochila.

$$ max \Sigma{Xi * Pi} $$
$$ \Sigma{Xi*Wi} <= C$$
$$ Xi \in{\{0,1\}} $$

Resolvendo com o Python MIP

In [4]:
!pip install mip

     |████████████████████████████████| 48.0 MB 50 kB/s 


In [5]:
from mip import Model, xsum, maximize, BINARY

p = [45,35,18,4,10,2]
w = [100,50,45,20,10,5]
c = 100
n = len(p)

In [6]:
# vamos definir o modelo
m = Model('Knapsack')


In [7]:
# para definir a funcao objetivo, precisamos das variaveis
x = [m.add_var(var_type=BINARY) for i in range(n)] # as variaveis tem tipo binario

#funcao objetivo
m.objective= maximize(xsum(x[i]*p[i] for i in range(n)))

#a unica restricao que temos
m += xsum(x[i] *w[i] for i in range(n)) <= c


In [8]:
m.optimize() #otimiza e busca pela solucao otima

<OptimizationStatus.OPTIMAL: 0>

A python mip usa um método que não tem precisão infinita, então resolve com uma determinada precisão para o ótimo.


In [9]:
print([x[i].x for i in range(n)])

[0.0, 1.0, 1.0, 0.0, 0.0, 1.0]


In [10]:
#os itens que ele selecionou sao os itens que tem valor >=.99 
# serve para resolver o problema da precisao da python mip para colocar valores reais

selected = [i for i in range(n) if x[i].x >= .99]
selected

[1, 2, 5]

[1,2,5] é a solução ótima desse problema. 

Podemos resolver esse problema utilizando força bruta também (ver brute force no meu github).

Para adicionar a restrição nesse caso, podemos fazer com que valores que infrinjam a restrição, se tornem as piores soluções, a assim possam ser ignoradas.



In [15]:
import numpy as np

def brute_force(fobj,n,m,partial_solution=[], best_solution=[], best_val=np.inf, print_sol=False):
  #np.inf = infinito
  
  
  #comeca com criterio de parada
  if len(partial_solution) ==n:
    fx = fobj(partial_solution) #coloquei o valor da ultima variavel em fx
    if print_sol:
      print('{} : {}'.format(partial_solution, fx))
    if fx <=best_val:
      best_solution = partial_solution
      best_val = fx
    return best_solution, best_val
  else:
    for e in set(np.arange(m)): #transforma em conjunto a partir do set para usar operacoes de conjuntos e descontar as repeticoes, tornando o algoritmo em permutacao
      best_sol, best_val = brute_force(fobj,n,m,partial_solution + [e], 
                  best_solution, 
                  best_val,
                  print_sol)
    return best_sol, best_val


In [18]:
#se nao entrar em nenhuma condicao, entao vai lançar no infinito. Isso fará que condições que violem as restrições, sempre percam, indo para infinito
#repare que uso -np.sum para utilizar como maximizacao, ja que o metodo usado é para minimizacao
fobj = lambda x: -np.sum([x[i]*p[i] for i in range(n)]) if np.sum([x[i]*w[i] for i in range(n)]) <= c else np.inf
best_solution,best_val = brute_force(fobj,n,2,print_sol=True)


[0, 0, 0, 0, 0, 0] : 0
[0, 0, 0, 0, 0, 1] : -2
[0, 0, 0, 0, 1, 0] : -10
[0, 0, 0, 0, 1, 1] : -12
[0, 0, 0, 1, 0, 0] : -4
[0, 0, 0, 1, 0, 1] : -6
[0, 0, 0, 1, 1, 0] : -14
[0, 0, 0, 1, 1, 1] : -16
[0, 0, 1, 0, 0, 0] : -18
[0, 0, 1, 0, 0, 1] : -20
[0, 0, 1, 0, 1, 0] : -28
[0, 0, 1, 0, 1, 1] : -30
[0, 0, 1, 1, 0, 0] : -22
[0, 0, 1, 1, 0, 1] : -24
[0, 0, 1, 1, 1, 0] : -32
[0, 0, 1, 1, 1, 1] : -34
[0, 1, 0, 0, 0, 0] : -35
[0, 1, 0, 0, 0, 1] : -37
[0, 1, 0, 0, 1, 0] : -45
[0, 1, 0, 0, 1, 1] : -47
[0, 1, 0, 1, 0, 0] : -39
[0, 1, 0, 1, 0, 1] : -41
[0, 1, 0, 1, 1, 0] : -49
[0, 1, 0, 1, 1, 1] : -51
[0, 1, 1, 0, 0, 0] : -53
[0, 1, 1, 0, 0, 1] : -55
[0, 1, 1, 0, 1, 0] : inf
[0, 1, 1, 0, 1, 1] : inf
[0, 1, 1, 1, 0, 0] : inf
[0, 1, 1, 1, 0, 1] : inf
[0, 1, 1, 1, 1, 0] : inf
[0, 1, 1, 1, 1, 1] : inf
[1, 0, 0, 0, 0, 0] : -45
[1, 0, 0, 0, 0, 1] : inf
[1, 0, 0, 0, 1, 0] : inf
[1, 0, 0, 0, 1, 1] : inf
[1, 0, 0, 1, 0, 0] : inf
[1, 0, 0, 1, 0, 1] : inf
[1, 0, 0, 1, 1, 0] : inf
[1, 0, 0, 1, 1, 1] : inf
[1, 0

In [20]:
best_solution, best_val
#a melhor solução tem o lucro de 55

([], -55)